In [1]:
import os
import h5py
from nltk.util import usage
import numpy as np
import datetime
import time
import warnings

from featureExtraction.tf.tf import *
from featureExtraction.tfidf.tfidf import *
from featureExtraction.fasttext.fasttext import *
from featureExtraction.lexicon.lexicon import *
from featureExtraction.orthography.orthography import *
from featureExtraction.pos.postag import *
from featureExtraction.tf.tf import *
from featureExtraction.word2vec.word2vec import *
from featureExtraction.ngram.ngram import *

from featureSelection.kFold import *

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
templated = [x for x in os.listdir("./Dataset/templated")]
templated.sort()

DATA_TEMPLATED = ["./Dataset/templated/" + x for x in os.listdir("./Dataset/templated")]
DATA_TEMPLATED.sort()
len_templated = len(DATA_TEMPLATED)

In [3]:
# print(DATA_TEMPLATED[4])

# df = pd.read_csv("./Dataset/templated/templated_posneg_Twitter_Emotion_Dataset.csv", sep="\;\;\;", engine="python", header=0, index_col=False)
# df.dropna(axis=0, inplace=True)

# df['label'].unique()

In [4]:
warnings.filterwarnings("ignore")
score_list = []
# for i in DATA_TEMPLATED[10:]:
count = 1
fail = 0
for i in DATA_TEMPLATED:
    print("Running {}/{}".format(count, len_templated), end=" ")
    try:
        # timer mulai
        result_fe, label, feat_name = ngram(i, 3)

        # create dataset
        # X, y = make_classification(n_samples=len(result_fe), n_features=len(result_fe[0]), random_state=1)
        # prepare the cross-validation procedure
        cv = KFold(n_splits=20, random_state=1, shuffle=True)
        x_train, x_test, y_train, y_test = train_test_split(result_fe, label)
              
        logRes = LogisticRegression(random_state=1, max_iter=1000)
        logRes.fit(x_train, y_train)
        predictions = logRes.predict(x_test)
        
        scores = cross_val_score(logRes, result_fe, label, scoring='f1_micro', cv=cv, n_jobs=-1)
        print('F1-score: %.3f (%.3f)' % (mean(scores), std(scores)))
        score_list.append(mean(scores))
        count += 1
        # timer selesai
    except Exception as e:
        count += 1
        fail += 1
        print(i, ": ", e)
        pass

print((sum(score_list)/len(score_list)))
print("Failed: {}".format(fail))

Running 1/24 F1-score: 0.660 (0.124)
Running 2/24 F1-score: 0.785 (0.087)
Running 3/24 F1-score: 0.863 (0.020)
Running 4/24 F1-score: 0.706 (0.062)
Running 5/24 ./Dataset/templated/templated_posneg_Twitter_Emotion_Dataset.csv :  Unable to allocate 3.42 GiB for an array with shape (4183, 109766) and data type int64
Running 6/24 